In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:

def find_columns(df, language):
    """Find text and label columns for a given language"""
    text_col = None
    label_col = None

    # Look for text columns
    possible_text_cols = [
        language,
        f"{language}_text",
        f"text_{language}",
        f"{language.capitalize()}",
        f"{language.upper()}"
    ]

    for col in possible_text_cols:
        if col in df.columns:
            text_col = col
            break

    possible_label_cols = [
        f"output_original_def_{language}",
        f"label_{language}",
        f"{language}_label",
        f"output_{language}",
        f"{language}_output",
        f"{language.capitalize()}_Label",
        "label",
        "output",
        "class"
    ]

    for col in possible_label_cols:
        if col in df.columns:
            label_col = col
            break

    return text_col, label_col


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/polish.csv")

df['id'] = range(len(df))

train_rows = []
eval_rows = []

for lang in ["english", "polish", "hindi"]:
    # Find columns
    text_col, label_col = find_columns(df, lang)
    if text_col is None or label_col is None:
        print(f"Skipping {lang}, missing columns")
        continue

    subset = df[['id', text_col, label_col]].copy()
    subset = subset.rename(columns={text_col: "text", label_col: "label"})
    subset["lang"] = lang

    label_map = {
        "english": {"yes": "Yes", "no": "No", 1: "Yes", 0: "No", "1": "Yes", "0": "No"},
        "polish": {"tak": "Tak", "nie": "Nie", 1: "Tak", 0: "Nie", "1": "Tak", "0": "Nie"},
        "hindi": {"हाँ": "हाँ", "नहीं": "नहीं", 1: "हाँ", 0: "नहीं", "1": "हाँ", "0": "नहीं"},
    }

    def normalize(label):
        str_label = str(label).strip().lower()
        return label_map[lang].get(str_label, None)

    subset["normalized_label"] = subset["label"].apply(normalize)
    subset = subset.dropna(subset=["normalized_label"])

    eval_lang = subset.groupby("normalized_label", group_keys=False).apply(
        lambda x: x.sample(min(len(x), 150), random_state=42)
    ).sample(frac=1, random_state=42)  # shuffle

    eval_rows.append(eval_lang)
    train_lang = subset.drop(index=eval_lang.index)
    train_rows.append(train_lang)

train_df = pd.concat(train_rows).sample(frac=1, random_state=42).reset_index(drop=True)
eval_df = pd.concat(eval_rows).sample(frac=1, random_state=42).reset_index(drop=True)

train_df.to_csv("/content/train_balanced.csv", index=False)
eval_df.to_csv("/content/eval_balanced.csv", index=False)

print(f"Saved {len(train_df)} training and {len(eval_df)} evaluation examples.")

print(f"Training set ID range: {train_df['id'].min()} - {train_df['id'].max()}")
print(f"Evaluation set ID range: {eval_df['id'].min()} - {eval_df['id'].max()}")
print(f"Total unique IDs in training: {train_df['id'].nunique()}")
print(f"Total unique IDs in evaluation: {eval_df['id'].nunique()}")

✅ Saved 4707 training and 900 evaluation examples.
Training set ID range: 0 - 1868
Evaluation set ID range: 10 - 1865
Total unique IDs in training: 1569
Total unique IDs in evaluation: 300


<ipython-input-4-2537471073>:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eval_lang = subset.groupby("normalized_label", group_keys=False).apply(
<ipython-input-4-2537471073>:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eval_lang = subset.groupby("normalized_label", group_keys=False).apply(
<ipython-input-4-2537471073>:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns

In [ ]:
df_train = pd.read_csv("/content/train_balanced.csv")
df_eval = pd.read_csv("/content/eval_balanced.csv")
print(df_train['lang'].value_counts())
print(df_eval['lang'].value_counts())

lang
english    1569
polish     1569
hindi      1569
Name: count, dtype: int64
lang
english    300
hindi      300
polish     300
Name: count, dtype: int64


In [ ]:
df_eval.head()

,id,text,label,lang,normalized_label
0,1283,So we’ve had authoritarianism since ’89 becaus...,0,english,No
1,1212,"{USERNAME}: जाहिर है, तुम्हें नहीं पता कि सबसे...",0,hindi,नहीं
2,601,American pussies can be sent to the slaughterh...,1,english,Yes
3,312,> {USERNAME}: spalić to w pizdu {USERNAME}: lg...,1,polish,Tak
4,1108,>You can’t impose additional duties (bearing a...,0,english,No


In [ ]:
df.to_csv("/content/polissh_ID.csv", index=False)

In [ ]:
df.head()

,Unnamed: 0,polish,english,hindi,label,id
0,0,{USERNAME}: Nadstawmy policzek do bicia po chr...,{USERNAME}: Let’s turn the other cheek like go...,{USERNAME}: ईसाई की तरह दूसरा गाल आगे करो और क...,1,0
1,1,"{USERNAME}: Spierdalaj. LGBT, to nie cyrk żeby...",{USERNAME}: Fuck off. LGBT isn’t a circus to p...,{USERNAME}: भाड़ में जाओ। LGBT कोई सर्कस नहीं ...,1,1
2,2,Zasrane mongolsko-ruzzkie ścierwo.,Fucking Mongol-Russian scum.,गंदा मंगोल-रूसी कचरा।,1,2
3,3,Jebać izrael,Fuck Israel.,इज़राइल को चोदो।,1,3
4,4,Najlepiej jakby ich wszystkich z Polszy tam wy...,It’d be best to send them all out of Poland. N...,सबसे अच्छा होगा अगर उन्हें पोलैंड से बाहर भेज ...,1,4


In [ ]:
import warnings
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

warnings.filterwarnings("ignore")

import torch
torch.cuda.empty_cache()

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import BitsAndBytesConfig, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import Dataset
import pandas as pd

model_id = "bigscience/bloomz-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)

#model with quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)

#k-bit training
print("Preparing model for k-bit training...")
model = prepare_model_for_kbit_training(model)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

#LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query_key_value"],
    bias="none"
)

model = get_peft_model(model, peft_config)
print("Trainable parameters:")
model.print_trainable_parameters()

# Load datasets
print("Loading datasets...")
train_df = pd.read_csv("/content/train_balanced.csv")
eval_df = pd.read_csv("/content/eval_balanced.csv")

# Examine dataset structure
print("Train dataset shape:", train_df.shape)
print("Eval dataset shape:", eval_df.shape)
print("\nColumn names:")
print(train_df.columns.tolist())
print("\nFirst few rows of train data:")
print(train_df.head())

# Definitions
definition = (
    "Definition of hate speech:\n"
    "\"Hate speech is any expression that offends, insults, or incites violence against a person or group "
    "based on their race, religion, nationality, sexual orientation, gender, or other characteristics.\""
)

language_examples = {
    "english": (
        'Example 1: "All Muslims are terrorists." → Yes\n'
        'Example 2: "I don\'t like this movie." → No'
    ),
    "polish": (
        'Przykład 1: "Wszyscy Żydzi to złodzieje." → Tak\n'
        'Przykład 2: "Nie lubię tej książki." → Nie'
    ),
    "hindi": (
        'उदाहरण 1: "सभी मुस्लिम आतंकवादी हैं।" → हाँ\n'
        'उदाहरण 2: "मुझे यह गाना पसंद नहीं है।" → नहीं'
    )
}

prompt_suffix = {
    "english": '\n\nText: "{text}"\nAnswer with "Yes" or "No":',
    "polish": '\n\nTekst: "{text}"\nOdpowiedz "Tak" lub "Nie":',
    "hindi": '\n\nपाठ: "{text}"\nउत्तर दें: "हाँ" या "नहीं":'
}

yesno_tokens = {
    "english": [" Yes", " No"],
    "polish": [" Tak", " Nie"],
    "hindi": [" हाँ", " नहीं"]
}

def process_dataset(df, dataset_name="dataset"):
    """Process a dataframe with the expected columns into training examples"""
    examples = []

    print(f"\n📊 Processing {dataset_name}...")

    # Language mapping for normalization
    lang_map = {
        "english": "english",
        "polish": "polish",
        "hindi": "hindi"
    }

    # Label mapping
    def normalize_label(label, lang):
        """Convert label to normalized form"""
        if label == 1 or str(label).strip() == "1":
            return {"english": "Yes", "polish": "Tak", "hindi": "हाँ"}[lang]
        elif label == 0 or str(label).strip() == "0":
            return {"english": "No", "polish": "Nie", "hindi": "नहीं"}[lang]
        else:
            # Handle text labels
            label_str = str(label).strip().lower()
            if label_str in ["yes", "tak", "हाँ"]:
                return {"english": "Yes", "polish": "Tak", "hindi": "हाँ"}[lang]
            elif label_str in ["no", "nie", "नहीं"]:
                return {"english": "No", "polish": "Nie", "hindi": "नहीं"}[lang]
            else:
                return None

    lang_counts = {}

    for _, row in df.iterrows():
        # Extract data from row
        text = str(row['text']).strip() if pd.notna(row['text']) else ""
        label = row['label']
        lang = str(row['lang']).strip().lower() if pd.notna(row['lang']) else ""

        # Skip if missing required data
        if not text or not lang or pd.isna(label):
            continue

        # Normalize language
        if lang not in lang_map:
            continue
        lang = lang_map[lang]

        # Normalize label
        normalized_label = normalize_label(label, lang)
        if normalized_label is None:
            continue

        # Count by language
        lang_counts[lang] = lang_counts.get(lang, 0) + 1

        # Create prompt
        prompt = f"{definition}\n{language_examples[lang]}{prompt_suffix[lang].format(text=text)}"
        full_text = f"{prompt} {normalized_label}"

        examples.append({
            "text": full_text,
            "label": normalized_label,
            "lang": lang,
            "prompt": prompt,
            "original_text": text,
            "original_label": label
        })

    print(f"   Added {len(examples)} examples")
    print(f"   Distribution by language: {lang_counts}")

    return examples

# Process both datasets
train_examples = process_dataset(train_df, "train dataset")
eval_examples = process_dataset(eval_df, "eval dataset")

print(f"\n Total training examples: {len(train_examples)}")
print(f"Total evaluation examples: {len(eval_examples)}")

# Check if we have data
if len(train_examples) == 0:
    print("\n No training examples were created.")
    raise ValueError("No training examples found. Please check the dataset structure.")

if len(eval_examples) == 0:
    print("\n No evaluation examples were created.")
    raise ValueError("No evaluation examples found. Please check the dataset structure.")

# Create datasets
train_dataset = Dataset.from_list(train_examples)
eval_dataset = Dataset.from_list(eval_examples)

print(f"\nTrain dataset created with {len(train_dataset)} examples")
print(f"Eval dataset created with {len(eval_dataset)} examples")

# Show distributions
print("\n Train set distribution:")
train_lang_counts = pd.Series([ex['lang'] for ex in train_examples]).value_counts()
print(f"By language: {train_lang_counts.to_dict()}")
train_label_counts = pd.Series([ex['label'] for ex in train_examples]).value_counts()
print(f"By label: {train_label_counts.to_dict()}")

print("\n Eval set distribution:")
eval_lang_counts = pd.Series([ex['lang'] for ex in eval_examples]).value_counts()
print(f"By language: {eval_lang_counts.to_dict()}")
eval_label_counts = pd.Series([ex['label'] for ex in eval_examples]).value_counts()
print(f"By label: {eval_label_counts.to_dict()}")

def tokenize(batch):
    texts = batch["text"] if isinstance(batch["text"], list) else [batch["text"]]

    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors=None
    )

    tokenized["labels"] = [ids.copy() for ids in tokenized["input_ids"]]
    return tokenized

print("\nTokenizing datasets...")
tokenized_train = train_dataset.map(
    tokenize,
    batched=True,
    batch_size=100,
    remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    tokenize,
    batched=True,
    batch_size=100,
    remove_columns=eval_dataset.column_names
)

print(f"Tokenized train set: {len(tokenized_train)} examples")
print(f"Tokenized eval set: {len(tokenized_eval)} examples")

# Clear cache before training
torch.cuda.empty_cache()

# Training arguments
training_args = TrainingArguments(
    output_dir="./bloomz-finetuned-hate",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-4,
    num_train_epochs=3,
    save_total_limit=1,
    fp16=True,
    bf16=False,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    report_to=[],
    logging_steps=25,
    disable_tqdm=False,
    gradient_checkpointing=False,
    dataloader_num_workers=0,
    warmup_steps=50,
    optim="adamw_torch",
    max_grad_norm=1.0,
    eval_accumulation_steps=2
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    return_tensors="pt",
    pad_to_multiple_of=8
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    processing_class=tokenizer,
    data_collator=data_collator
)

# Clear cache before training
torch.cuda.empty_cache()

# VERIFICATION: Check that LoRA parameters have requires_grad=True
print("\n🔍 Checking LoRA parameters:")
trainable_params = 0
total_params = 0
for name, param in model.named_parameters():
    total_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        if 'lora' in name.lower():
            print(f" {name}: requires_grad={param.requires_grad}")

print(f"\nTrainable params: {trainable_params:,}")
print(f"Total params: {total_params:,}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")

if trainable_params == 0:
    print(" ERROR: No trainable parameters found!")
    raise ValueError("No trainable parameters - training will not work")

print("\n🚀 Starting training...")
try:
    trainer.train()
    print(" Training completed successfully!")
except RuntimeError as e:
    if "out of memory" in str(e):
        print(" OOM error. Try reducing batch size or max_length further.")
        raise e
    else:
        print(f" Training error: {e}")
        raise e

print("\n📊 Evaluating model...")

from torch.nn.functional import log_softmax
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import gc

prompts = [ex["prompt"] for ex in eval_examples]
labels = [ex["label"] for ex in eval_examples]
langs = [ex["lang"] for ex in eval_examples]
preds, logit_yes_list, logit_no_list = [], [], []

batch_size = 4
for i in tqdm(range(0, len(prompts), batch_size), desc="Evaluating"):
    batch_prompts = prompts[i:i+batch_size]
    batch_langs = langs[i:i+batch_size]

    for prompt, lang in zip(batch_prompts, batch_langs):
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(model.device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits[:, -1, :]
            log_probs = log_softmax(logits, dim=-1)

            yes_token_id = tokenizer(yesno_tokens[lang][0], add_special_tokens=False)["input_ids"][0]
            no_token_id = tokenizer(yesno_tokens[lang][1], add_special_tokens=False)["input_ids"][0]

            score_yes = log_probs[0, yes_token_id].item()
            score_no = log_probs[0, no_token_id].item()

            pred_label = yesno_tokens[lang][0].strip() if score_yes > score_no else yesno_tokens[lang][1].strip()
            preds.append(pred_label)
            logit_yes_list.append(score_yes)
            logit_no_list.append(score_no)

    torch.cuda.empty_cache()
    gc.collect()

acc = accuracy_score(labels, preds)
print(f"\n Overall Accuracy: {acc:.4f}")

# Calculate accuracy by language
languages_in_eval = list(set(langs))
for lang in languages_in_eval:
    lang_mask = [l == lang for l in langs]
    if any(lang_mask):
        lang_labels = [labels[i] for i, mask in enumerate(lang_mask) if mask]
        lang_preds = [preds[i] for i, mask in enumerate(lang_mask) if mask]
        lang_acc = accuracy_score(lang_labels, lang_preds)
        print(f" {lang.capitalize()} Accuracy: {lang_acc:.4f}")

# Save results
results_df = pd.DataFrame({
    "prompt": prompts,
    "ground_truth": labels,
    "prediction": preds,
    "language": langs,
    "logit_yes": logit_yes_list,
    "logit_no": logit_no_list
})

results_df.to_csv("finetuned_bloomz_predictions_with_logits.csv", index=False)
print(f"\n Saved {len(results_df)} predictions to 'finetuned_bloomz_predictions_with_logits.csv'")

# Show confusion matrix info
print("\nPrediction distribution:")
print(pd.Series(preds).value_counts())
print("\nGround truth distribution:")
print(pd.Series(labels).value_counts())

print("\n Training and evaluation completed!")

tokenizer_config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

...

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,1.219300,1.133359
2,1.061500,1.108967
3,1.073400,1.111909


✅ Training completed successfully!

📊 Evaluating model...


Evaluating: 100%|██████████| 225/225 [03:35<00:00,  1.05it/s]


✅ Overall Accuracy: 0.8374
✅ English Accuracy: 0.8528
✅ Hindi Accuracy: 0.8997
✅ Polish Accuracy: 0.7600

✅ Saved 898 predictions to 'finetuned_bloomz_predictions_with_logits.csv'

Prediction distribution:
Tak     168
Yes     159
हाँ     153
नहीं    146
No      140
Nie     132
Name: count, dtype: int64

Ground truth distribution:
No      150
नहीं    150
Tak     150
Nie     150
Yes     149
हाँ     149
Name: count, dtype: int64

🎉 Training and evaluation completed!
